### Customers who bought '[this item]' also bought '[that item]':
1. Description  : Customers who bought this item also bought these items.
2. Scenario     : Bought,Bought
3. field name   : BoughtBoughtProducts
4. Code Strategy: pure CF(bought)



In [ ]:
def model_(df1, subject):
    df = df1.copy()
    from lightfm.data import Dataset
    dataset = Dataset()
    dataset.fit(users=df[subject].unique(), items=df.ProductId.unique())
    (interactions, weights) = dataset.build_interactions([(x[subject], x['ProductId'], x['ratings'])
                                                          for index, x in df.iterrows()])
    
    from lightfm import LightFM
    model = LightFM(loss='warp')
    model.fit(interactions, sample_weight=weights)
    return model

def id_mappings(user_list, item_list,customer_list):
    '''Make the list of dictionary and map with indices to get used'''
    dict_lst =[]
    for lst in [user_list,item_list,customer_list]:
        dict_lst.append({_id:_index for _index, _id in enumerate(lst)})
        dict_lst.append({_index:_id for _index, _id in enumerate(lst)})
        
    return dict_lst[0], dict_lst[1], dict_lst[2], dict_lst[3],dict_lst[4]


def recommendation_dataframe(to_recommend, model, subject, products):
        (recommend_count, name_target) = (50, "userTargetProducts") if subject == 'UserId' \
            else (200, "customerTargetProducts")
        usr_dict_pdoct = {}
        for user in to_recommend:
            RecScore = model.predict(user_ids=user, item_ids=products)
            top_items = products[np.argsort(-RecScore)][:recommend_count]
    #             top_items = [productId_dict_Guid[p] for p in top_items]
            usr_dict_pdoct.update({user: list(top_items)})

        df_recommend = pd.DataFrame(list(usr_dict_pdoct.items()), columns=[subject, name_target])
        return df_recommend

#### We need to consider User and Customer behavior and merge them as shown bellow:
finally we need 20 products to recommend individual user

In [100]:
def user_products_filtering(customer_to_recommend,products_to_recommend,customer_model,customer,user_Rcommend):
    df_list = []

    dframe_length = len(customer_to_recommend)
    rows = dframe_length if dframe_length < 500 else dframe_length // 10
    for row1 in range(0, dframe_length, rows):
        subjectSlice = customer_to_recommend[row1:dframe_length] if \
            (row1 + rows) > dframe_length else customer_to_recommend[row1:(row1 + rows)]
        customer_Rcommendation_slice = recommendation_dataframe(subjectSlice, customer_model, customer,
                                                                products_to_recommend)
        recommend_slice = user_Rcommend.merge(customer_Rcommendation_slice, on=customer)
        recommend_slice['TargetProducts'] = recommend_slice.swifter.progress_bar(False).apply(
            lambda row: {*row['userTargetProducts']} & {*row['customerTargetProducts']}, axis=1)
        recommend_slice['customerTargetProducts'] = recommend_slice.swifter.progress_bar(False).apply(
            lambda row: ({*row['customerTargetProducts']} - row['TargetProducts']), axis=1)
        recommend_slice['TargetProducts'] = recommend_slice['TargetProducts'].swifter.progress_bar(
            False).apply(list)
        recommend_slice['customerTargetProducts'] = recommend_slice['customerTargetProducts'].swifter.progress_bar(
            False).apply(list)
        recommend_slice['TargetProducts'] = recommend_slice.swifter.progress_bar(False).apply(lambda row: (
                row['TargetProducts'] + (row['customerTargetProducts'])[
                                              :(20 - len(row['TargetProducts']))]) if len(
            row['TargetProducts']) < 20 else row['TargetProducts'], axis=1)
        recommend_slice.drop(['userTargetProducts', 'customerTargetProducts', customer], axis=1, inplace=True)
        df_list.append(recommend_slice)
    whole_df = pd.concat(df_list)
    return whole_df

## How to save and load model

In [142]:
import pickle
def save_model(model, model_name, model_collection):
    from datetime import datetime as dt
    pickled_model = pickle.dumps(model)  # pickling the model
    mycon = db[model_collection]
    mycon.delete_one({'name': model_name})
    info = mycon.insert_one({model_name: pickled_model, 'name': model_name, 'trained_on':str(dt.utcnow().date())})
    print(info.inserted_id, ' saved with this id successfully!')
#     details = {'model_name':model_name, 'inserted_id': info.inserted_id, 'trained_on': str(dt.utcnow().date())}
    return 
  
def load_saved_model(model_name, model_collection):
    conn = db[model_collection]
    data = conn.find({'name': model_name})
    for i in data:
        json_data = i
    pickled_model = json_data[model_name]
    return pickle.loads(pickled_model) 

In [143]:
def create_and_save_model(df1, webGuid,user,customer):
    df = df1.copy()        
    user_model = model_(ratings(df, user), user)
    customer_model = model_(ratings(df, customer), customer)
    save_model(model = user_model, model_collection = 'model_for_recommend', model_name = 'modelUser_{0}'.format(guid_dict_webName[webGuid]))
    save_model(model = customer_model, model_collection = 'model_for_recommend', model_name = 'modelCustomer_{0}'.format(guid_dict_webName[webGuid]))

### Prepare product order data with the bellow mentioned fields which has integer values as Id
dframe_model consist of following columns:
'UserGuid', 'ProductGuid', 'CustomerGuid','ratings'

In [ ]:
import pandas as pd
df_users,df_products,df_customers = (dframe_model.UserGuid.unique(),dframe_model.ProductGuid.unique(),dframe_model.CompanyName.unique())

userGuid_dict_Id,userId_dict_Guid, productGuid_dict_Id,productId_dict_Guid, CompanyName_dict_Id = \
id_mappings(df_users,df_products,df_customers)

dframe_model['UserId'] = dframe_model['UserGuid'].map(userGuid_dict_Id)
dframe_model['ProductId'] = dframe_model['ProductGuid'].map(productGuid_dict_Id)
dframe_model['CustomerId'] = dframe_model['CompanyName'].map(CompanyName_dict_Id)
del dframe_model['UserGuid']
del dframe_model['ProductGuid']
del dframe_model['CompanyName']    

Now, we got dataFrame which is desire for lightFM model building input 

In [86]:
users_to_recommend,customers_to_recommend,products_to_recommend= \
dframe_model.UserId.unique(), dframe_model.CustomerId.unique(),dframe_model.ProductId.unique()

user, customer = "UserId", "CustomerId"

### Now, create the model and save into mongoDB

In [146]:
create_and_save_model(dframe_model,webGuid,user,customer)

ratings function took 0.001 min
User Featutes None
Item Featutes None
ratings function took 0.001 min
User Featutes None
Item Featutes None
5e1420dca13218bdd9fbf7f4  saved with this id successfully!
5e1420dda13218bdd9fbf7f5  saved with this id successfully!


### you can retrieve model from mongoDB using following function

In [129]:
user_model  = load_saved_model(model_name = 'modelUser_{0}'.format(guid_dict_webName[webGuid]), model_collection = 'model_for_recommend')
customer_model  = load_saved_model(model_name = 'modelCustomer_{0}'.format(guid_dict_webName[webGuid]), model_collection = 'model_for_recommend')

### get the product recommnedatin based on model built 

In [ ]:
user_Rcommendation = recommendation_dataframe(users_to_recommend, user_model, user, products_to_recommend)

#### Regain Guid values:

In [ ]:
user_Rcommendation['UserGuid'] = user_Rcommendation['UserId'].map(userId_dict_Guid)

user_Rcommendation['ModelProducts'] = user_Rcommendation['userTargetProducts'].swifter.progress_bar(False)\
.apply(lambda row: [productId_dict_Guid.get(product) for product in row]) 
del user_Rcommendation['UserId']
del user_Rcommendation['userTargetProducts']

In this way you will see the actual values of product and user which is defined in original order dataframe

In [ ]:
user_Rcommendation.head()